<a href="https://colab.research.google.com/github/thisisquangphuc/fhdo-distributed-system/blob/issac/Truckplatooning_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cupy as cp
import numpy as np
import concurrent.futures

# Step 1: GPU Obstacle Detection and Avoidance
def obstacle_detection_and_avoidance(truck_positions, obstacle_positions, threshold, avoidance_factor):
    """
    Detect obstacles and calculate new directions for avoidance.

    Args:
    - truck_positions: List of truck positions [[x, y], ...]
    - obstacle_positions: List of obstacle positions [[x, y], ...]
    - threshold: Safe distance threshold
    - avoidance_factor: Factor to determine avoidance direction

    Returns:
    - new_positions: Updated positions for trucks [[x, y], ...].
    """
    truck_array = cp.array(truck_positions.tolist(), dtype=cp.float32)  # Truck positions on GPU
    obstacle_array = cp.array(obstacle_positions.tolist(), dtype=cp.float32)  # Obstacle positions on GPU

    # Calculate pairwise distances between trucks and obstacles
    distances = cp.sqrt(cp.sum((truck_array[:, None, :] - obstacle_array[None, :, :]) ** 2, axis=2))

    # Find the nearest obstacle for each truck
    min_distances = cp.min(distances, axis=1)
    closest_obstacles = obstacle_array[cp.argmin(distances, axis=1)]

    # Determine if avoidance is needed
    avoid_mask = min_distances < threshold

    # Calculate avoidance directions for trucks that need to avoid
    avoid_directions = truck_array + avoidance_factor * (truck_array - closest_obstacles)

    # Update positions: avoid for trucks under threshold, retain original positions otherwise
    new_positions = cp.where(avoid_mask[:, None], avoid_directions, truck_array)

    return new_positions.get()  # Return to CPU

# Step 2: Synchronization (Logical Matrix Clock)
def synchronization(clock_matrix, my_id, sender_id):
    clock_gpu = cp.array(clock_matrix.tolist(), dtype=cp.int32)

    clock_gpu[my_id, my_id] += 1
    clock_gpu[my_id] = cp.maximum(clock_gpu[my_id], clock_gpu[sender_id])

    return clock_gpu.get()

# Step 3: Multi-threaded Truck Processing
def process_chunk(truck_chunk, obstacle_positions, threshold, avoidance_factor, clock_matrix, my_id, sender_id):
    """
    Process a subset of trucks: detect obstacles, calculate avoidance, and synchronize logical clocks.

    Args:
    - truck_chunk: Subset of truck positions
    - obstacle_positions: List of obstacle positions
    - threshold: Safe distance threshold
    - avoidance_factor: Factor for avoidance calculation
    - clock_matrix: Logical clock matrix
    - my_id: Thread ID of the current chunk
    - sender_id: Thread ID of the sender for synchronization

    Returns:
    - updated_positions: New positions for the trucks in the chunk
    - updated_clock_matrix: Updated logical clock matrix
    """
    # Obstacle detection and avoidance
    updated_positions = obstacle_detection_and_avoidance(truck_chunk, obstacle_positions, threshold, avoidance_factor)

    # Synchronize clocks
    updated_clock_matrix = synchronization(clock_matrix, my_id, sender_id)

    return updated_positions, updated_clock_matrix

def main():
    # Simulated truck and obstacle data
    truck_positions = np.random.rand(1000, 2) * 1000  # 1000 trucks
    obstacle_positions = np.random.rand(100, 2) * 1000  # 100 obstacles
    threshold = 50.0
    avoidance_factor = 1.5
    num_threads = 4

    # Initialize logical clock matrix
    clock_matrix = np.eye(num_threads, dtype=int)

    # Divide truck positions into chunks for threading
    chunk_size = len(truck_positions) // num_threads
    chunks = [truck_positions[i:i + chunk_size] for i in range(0, len(truck_positions), chunk_size)]

    results = []
    updated_clocks = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [
            executor.submit(
                process_chunk,
                chunk,
                obstacle_positions,
                threshold,
                avoidance_factor,
                clock_matrix,
                thread_id,
                (thread_id + 1) % num_threads
            )
            for thread_id, chunk in enumerate(chunks)
        ]

        for future in concurrent.futures.as_completed(futures):
            updated_positions, updated_clock = future.result()
            results.append(updated_positions)
            updated_clocks.append(updated_clock)

    # Combine results from all threads
    final_positions = np.vstack(results)
    final_clock_matrix = np.sum(updated_clocks, axis=0)

    # Display results
    print("Final Updated Truck Positions (first 10):")
    print(final_positions[:10])
    print("\nFinal Logical Clock Matrix:")
    print(final_clock_matrix)

if __name__ == "__main__":
    main()


Final Updated Truck Positions (first 10):
[[  51.841896  365.8621  ]
 [ 766.2256    683.43066 ]
 [ 823.4142    849.6599  ]
 [ 157.19223   836.3401  ]
 [ 601.8194    425.9489  ]
 [ 319.9608    200.02724 ]
 [ 661.3073    374.13782 ]
 [ 598.0548    469.43515 ]
 [ 923.94415   144.81317 ]
 [1014.43726   971.4091  ]]

Final Logical Clock Matrix:
[[5 1 0 0]
 [0 5 1 0]
 [0 0 5 1]
 [1 0 0 5]]
